In [2]:
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

/Users/ips/Documents/trail/AI Agents/aiagent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## Vanilla Chatbot
## Problems - Long conversation history,results in high token usage and slow response times and cost

#Set the system message
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Explain the concept of AI?"),
    AIMessage(content="AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.")
]

while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    messages.append(HumanMessage(content=user_input))
    print(messages)
    response = llm.invoke(messages)
    messages.append(AIMessage(content=response.content))
    print(response.content)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the concept of AI?', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='test this AI', additional_kwargs={}, response_metadata={})]
Okay, I'm ready to be tested! What would you like to test me on? To give you the most helpful response, please be specific. For example, you could test me on:

*   **My knowledge:** Ask me questions about a specific topic.
*   **My reasoning abilities:** Give me a problem to solve or a scenario to analyze.
*   **My creative writing skills:** Ask me to write a poem, story, or script.
*   **My ability to follow instructions:** Give me a set of instr

In [4]:
## Filtering
# Same code just top 3 messages with system message always included

#Set the system message
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Explain the concept of AI?"),
    AIMessage(content="AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.")
]

while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    messages.append(HumanMessage(content=user_input))
    print("Full messages", messages)

    try:
        # Always include system message plus last 2 messages
        filtered_messages = [messages[0]] + messages[-2:]
        print("Filtered messages", filtered_messages)
        response = llm.invoke(filtered_messages)
        messages.append(AIMessage(content=response.content))
        print(response.content)
    except Exception as e:
        print("Error:", e)
        response = llm.invoke(messages)  # Fallback to full history
        messages.append(AIMessage(content=response.content))
        print(response.content)

Full messages [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the concept of AI?', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='test ai', additional_kwargs={}, response_metadata={})]
Filtered messages [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='test ai', additional_kwargs={},

In [5]:
# Trimming
# Same code just trim last n words or tokens

# Same code trimmed to last 100 words of conversation with system message

#Set the system message
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

def get_last_n_words(messages, n):
    total_words = 0
    trimmed_messages = []
    # Iterate over the messages in reverse order (excluding system message)
    for msg in reversed(messages[1:]):
        words = msg.content.split() # Split the content into words
        if total_words + len(words) > n:
            words_to_take = n - total_words
            if words_to_take > 0:
                trimmed_content = ' '.join(words[-words_to_take:])
                if isinstance(msg, HumanMessage):
                    trimmed_messages.insert(0, HumanMessage(content=trimmed_content))
                elif isinstance(msg, AIMessage):
                    trimmed_messages.insert(0, AIMessage(content=trimmed_content))
                total_words += words_to_take
            break
        else:
            trimmed_messages.insert(0, msg)
            total_words += len(words)
        if total_words >= n:
            break
    return trimmed_messages

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Explain the concept of AI?"),
    AIMessage(content="AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.")
]

while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    messages.append(HumanMessage(content=user_input))
    print("Full messages", messages)

    try:
        # Trim the conversation to the last 100 words
        trimmed_conversation = get_last_n_words(messages, 100)
        # Always include the system message
        filtered_messages = [messages[0]] + trimmed_conversation
        print("Filtered messages", filtered_messages)
        response = llm.invoke(filtered_messages)
        messages.append(AIMessage(content=response.content))
        print(response.content)
    except Exception as e:
        print("Error:", e)
        response = llm.invoke(messages)  # Fallback to full history
        messages.append(AIMessage(content=response.content))
        print(response.content)

Full messages [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the concept of AI?', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is AI', additional_kwargs={}, response_metadata={})]
Filtered messages [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the concept of AI?', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.',